In [1]:
import pandas as pd
import numpy as np
import warnings
import re

import requests
import play_scraper
import time

from uszipcode import SearchEngine

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
import statsmodels.formula.api as sm

#nltk.downloader.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
warnings.filterwarnings("ignore")

Note: Some cells have been pre-run in other notebooks

## Data Extraction and Munging

### Apple Store Data

####  Apple Data Cleaning
We first download apple dataset from a [kaggles dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps). 
- data_apple: Apple datasets from Kaggles

In [2]:
#1. Data Extraction and Munging
#1.1 Apple Store Data
#1.1.1 Apple Data Cleaning

#read the data, and description of apps
data_apple = pd.read_csv('../data/AppleStore.csv')
des_apple = pd.read_csv('../data/appleStore_description.csv') 

#drop unrelated columns, rename column name , fill 'Source' and 'Type', add description to each app
data_apple.drop(['Unnamed: 0', 'currency','vpp_lic'], axis=1, inplace=True)
data_apple.columns.values[[1,2,3,6,8,9,10]] = ['Name','Size','Price','Rating','Version','Content_Rating','Category']
data_apple['Size'] = data_apple['Size']/(1024*1024)
data_apple['Source'] = "Apple"
data_apple['Type'] = data_apple['Price'].apply(lambda x: 'Paid' if x > 0 else 'Free')
data_apple = pd.merge(data_apple, des_apple[['id','app_desc']], on='id')
data_apple['id'] = data_apple['id'].astype('str')
data_apple.head(2)

,id,Name,Size,Price,rating_count_tot,rating_count_ver,Rating,user_rating_ver,Version,Content_Rating,Category,sup_devices.num,ipadSc_urls.num,lang.num,Source,Type,app_desc
0,281656475,PAC-MAN Premium,96.119141,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,Apple,Paid,"SAVE 20%, now only $3.99 for a limited time!\n..."
1,281796108,Evernote - stay organized,151.232422,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,Apple,Free,Let Evernote change the way you organize your ...


#### Apple Data Scrapping
We are also intrested what people would say in thier App reviews. We expected to get some useful positive and negative feedback. From the user feedbacks, developers can evaluate thier App and understand what they should keep, avoid and make change or improvement. 

Since Apple only release top 50 reviews by review categories, for each App, we scraped 50 most helpful reviews which defined by iTunes. Note that some App does not have any review.
- apple_review: Scraped review of app in Apple from the iTunes RSS Genearator API.

In [ ]:
#1.1.2 Apple Data Cleaning
#App scraping function
def apple_review(js):
    """
    Scrape the reviews from iTunes' review api and append to given lists
    -----
    Argument:
        js: Get Json file
        url: Url of the link.
    """
    try:
        var = js['feed']['entry']
        for i in range(len(var)):
            appid.append(re.search('id=(.+)/s', url).group(1))
            title.append(var[i]['title']['label'])
            rating.append(var[i]['im:rating']['label'])
            review.append(var[i]['content']['label'])
    except KeyError:
        return None

In [ ]:
#create new lists for storing data
appid = []
title = []
rating = []
review = []
#scarpping review data
for i in data_apple.id[0:7197]:
    url = 'https://itunes.apple.com/rss/customerreviews/id={}/sortBy=mostHelpful/json'.format(i)
    req = requests.get(url)
    js = req.json()
    apple_review(js)

In [ ]:
#save the file to local
cols = ['appid','title','rating','review']
df = pd.DataFrame(list(zip(appid,title,rating,review)), columns = cols)
#converting dataframe into csv
df.to_csv('../data/apple_review.csv',encoding='utf_8_sig')

In [ ]:
#reload the itunes review from prior saved data
review_apple = pd.read_csv('../data/apple_review.csv',low_memory=False)
review_apple = review_apple.drop('Unnamed: 0', 1)
review_apple.head(2)

#### Apple Data of User Review
We tried the sentitment analysis for user review using existing pacakge in order to predict the rating in the further step. We are also curious about the relationship between word length and user rating. The dataset for Apple review has the features of sentiment analysis for both title and review, and the word length for each title and review.

In [3]:
#1.1.3 Apple Data Sentiment Analysis
#original data of apple review
review_apple = pd.read_csv('../data/apple_review.csv')
# drop NA
review_apple.dropna(subset=['title'], inplace=True)
review_apple = review_apple.reset_index(drop = True)
# create two new columns 
review_apple['Title Compound'] = 0
review_apple['Review Compound'] = 0

In [ ]:
#do sentiment analysis, and saved it to a new csv file apple_review2
#nltk.downloader.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
for i in range(apple_review.shape[0]):
    apple_review['Title Compound'].iloc[i] = sid.polarity_scores(apple_review['title'][i])['compound']
    apple_review['Review Compound'].iloc[i] = sid.polarity_scores(apple_review['review'][i])['compound']
review_apple.to_csv('apple_review2.csv')

- apple_review2: add sentiment analysis, title length, and review length based on review_apple

In [4]:
#get apple_review2 with sentiment analysis
apple_review2 = pd.read_csv('../data/apple_review2.csv')
apple_review2 = apple_review2.drop('Unnamed: 0', 1)
apple_review2.rename(columns={'Title Compound': 'title_comp','Review Compound':'review_comp'}, inplace=True)
apple_review2.drop_duplicates(['review'],inplace = True)
apple_review2 = apple_review2[(apple_review2['title_comp'] != 0) | (apple_review2['review_comp'] != 0)]

# Calculate the word length
apple_review2['title length'] = apple_review2['title'].str.split()
apple_review2['title length'] = apple_review2['title length'].apply(len)

apple_review2['review length'] = apple_review2['review'].str.split()
apple_review2['review length'] = apple_review2['review length'].apply(len)

apple_review2.head(2)

,appid,title,rating,review,title_comp,review_comp,title length,review length
0,281796108.0,Come back in - the water is fine,5.0,Having been a long time and premium user of Ev...,0.2023,0.7690,8,248
1,281796108.0,Customer Service,1.0,If you are looking for a company that has acce...,0.0000,0.9892,2,247


### Google Play Data
We first download apple dataset from a [kaggles dataset](https://www.kaggle.com/lava18/google-play-store-apps). However, this dataset only has a small fraction overlap with our Apple dataset, so we scrape more App information using [`play-scraper`](https://pypi.org/project/play-scraper/) based on the name of Apple. 

#### Google Play Data Scrapping

In [ ]:
#1.2 Google Play Data
#1.2.1 Google Play Data Scrapping

#creating empty lists to store data
Name = []
Appid = []
Rating =[]
sc_num = []
Price = []
Reviews = []
Description = []
Category = []
Version = []
Size = []
Update_time = []
Installs = []
Content_rating = []
Req_ver = []
Edit_choice = []
IAP = []
IAP_range = []
Rate_hist = []
developer_address=[]
iosname = []

In [ ]:
#scrape App information from google play store based on Apple dataset
def android_app(name):
    '''
    Scrape App information from google play store
    -----
    Argument:
        name: input name in the search engine 
    '''
    search = play_scraper.search(name)
    try:
        app = play_scraper.details(search[0]['app_id'])
        Name.append(app['title'])
        Appid.append(app['app_id'])
        Rating.append(app['score'])
        sc_num.append(len(app['screenshots']))
        Price.append(app['price'])
        Reviews.append(app['reviews'])
        Description.append(app['description'])
        Category.append(app['category'])
        Version.append(app['current_version'])
        Size.append(app['size'])
        Update_time.append(app['updated'])
        Installs.append(app['installs'])
        Content_rating.append(app['content_rating'])
        Req_ver.append(app['required_android_version'])
        Edit_choice.append(app['editors_choice'])
        IAP.append(app['iap'])
        IAP_range.append(app['iap_range'])
        Rate_hist.append(app['histogram'])
        developer_address.append(app['developer_address'])
        iosname.append(name)
    except:
        return None

In [ ]:
#data scraping
start_time = time.time()
for names in data_apple.Name:
    android_app(names)
    time.sleep(0.1)
print(time.time() -start_time)

In [ ]:
#store data into dataframe
cols_a= ['Name','Appid','Rating','sc_num','Price','Reviews','Description'\
        ,'Category','Version','Size','Update_time','Installs'\
        ,'Content_rating','Req_ver','Edit_choice','IAP','IAP_range'\
        ,'Rate_hist','developer_address','iosname']

df_a = pd.DataFrame(list(zip(Name,Appid,Rating,sc_num,Price,Reviews,Description\
        ,Category,Version,Size,Update_time,Installs\
        ,Content_rating,Req_ver,Edit_choice,IAP,IAP_range\
        ,Rate_hist,developer_address,iosname)), columns = cols_a)
#converting dataframe into csv
#df_a.to_csv('../data/new_android.csv',encoding='utf_8_sig')

- data_android2: data scrapped based on the name of the data_apple dataset

In [5]:
#reload the dataset: scraped data based on names in Apple datasets
data_android2 = pd.read_csv('../data/new_android.csv')
data_android2.head(2)

,Unnamed: 0,Name,iosname,name_similarity,Appid,Rating,sc_num,Price,Reviews,Description,...,Size,Update_time,Installs,Content_rating,Req_ver,Edit_choice,IAP,IAP_range,Rate_hist,developer_address
0,0,PAC-MAN Championship Edition DX,PAC-MAN Premium,0.434783,com.bandainamcoent.pacmancedx,4.5,6,$1.99,1816,New Price – For a Limited Time Only!\nGet read...,...,36M,"April 23, 2018","10,000+",['Everyone'],4.0.3 and up,False,False,NaN,"{5: 1347, 4: 251, 3: 83, 2: 36, 1: 99}","2051 Mission College Blvd., Santa Clara, CA 9..."
1,1,Evernote,Evernote - stay organized,0.484848,com.evernote,4.5,21,0,1497796,Evernote helps you focus on what matters most ...,...,Varies with device,"December 1, 2018","100,000,000+",['Everyone'],5.0 and up,True,True,"('$1.00', '$99.99')","{5: 1071594, 4: 295588, 3: 58942, 2: 21822, 1:...","305 Walnut Street\nRedwood City, CA 94063"


In [6]:
#function for data cleaning
def covert_size(size):
    '''
    convert the android size to MB
    -----
    Argument:
        size: original string size
    Output:
        size in MB
    '''
    if size == 'Varies with device':
        return np.nan
    elif size.endswith('M'):
        size = size[:-1]
        return float(size)
    elif size.endswith('K'):
        size = size[:-1]
        return float(size)/0.001
    
def compare(str1,str2):
    '''
    compare the name of Apple and Android, check whether thier first words are the same 
    -----
    Argument:
        str1: input compared string
        str2: input compared string
    Output:
        True: thier first words are the same
        False: thier first words are not the same 
    '''
    a = re.split(r"[\W']+",str1.lower())
    b = re.split(r"[\W']+",str2.lower())
    if a[0]==b[0]:
        return True
    else:
        return False

In [7]:
#data_android1
data_android1 = pd.read_csv('../data/googleplaystore.csv')
#remove row with error 
data_android1.drop(index = 10472,inplace = True)
#assign column names
data_android1.columns =['Name', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',\
       'Price', 'Content_rating', 'Genres', 'Update_time', 'Version',\
       'Req_ver']
#change price to numeric
data_android1['Price'] = data_android1['Price'].apply(lambda x: float(x.strip('$')))
#clean size
data_android1['Size'] = data_android1['Size'].apply(covert_size)

- data_android2: Scraped data based on names in Apple datasets

In [8]:
#data_android2
data_android2 = pd.read_csv('../data/new_android.csv')
data_android2 = data_android2.drop('Unnamed: 0', 1)  
data_android2['name_similarity'] = [compare(data_android2.Name[i],data_android2.iosname[i])\
                                    for i in range(len(data_android2))]
#clean category and match to data_android1
data_android2.Category = data_android2.Category.apply(lambda x: re.split(r"[\W']+",x)[1])
game_cat = data_android2[data_android2.Category.str.contains('GAME')==True].Category.unique()
data_android2.Category.replace(game_cat,'GAME',inplace=True)
#match content rating to data_android1
data_android2.Content_rating=data_android2.Content_rating.apply(lambda x: re.split(r"[\W']+",x)[1]\
                                                                +' '+re.split(r"[\W']+",x)[2]+'+' \
                                   if (re.split(r"[\W']+",x)[2]=='10' or re.split(r"[\W']+",x)[2]=='17')\
                                                                else re.split(r"[\W']+",x)[1])
#clean prices data
data_android2['IAP'] = data_android2['IAP'].apply(lambda x: 'Paid' if x == True else 'Free')
data_android2['Price'] = data_android2['Price'].apply(lambda x: float(x.strip('$')))
data_android2['Type'] = data_android2['Price'].apply(lambda x: 'Paid' if x > 0 else 'Free')
#clean size
data_android2['Size'] = data_android2['Size'].apply(covert_size)
data_android2.shape

(4950, 22)

- data_android: Combine both Google play data sources for Android Market analysis

In [9]:
#combine both google play data sources
data_android = pd.concat([data_android1, data_android2], join='inner')
data_android.drop_duplicates(['Name'],inplace = True)

data_android['Type'] = data_android['Price'].apply(lambda x: 'Paid' if x > 0 else 'Free')
#claen variable formats
data_android['Installs'] = data_android['Installs'].apply(lambda x: float(x.replace(',','').strip('+')))
data_android['Reviews'] = data_android.Reviews.astype('float')
data_android['Update_time'] = pd.to_datetime(data_android['Update_time'], format='%B %d, %Y')
data_android['Category'] = data_android['Category'].str.replace('_',' ').str.title()
#match content rating
data_android['Content_rating'] = data_android['Content_rating'].replace('Adults only 18+','Mature 17+')
data_android['Content_rating'] = data_android['Content_rating'].apply(lambda x: np.nan if x =='Unrated' else x)
data_android.rename(columns={'Content_rating': 'Content_Rating'}, inplace=True)
data_android.head(2)

,Name,Category,Rating,Reviews,Size,Installs,Type,Price,Content_Rating,Update_time,Version,Req_ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art And Design,4.1,159.0,19.0,10000.0,Free,0.0,Everyone,2018-01-07,1.0.0,4.0.3 and up
1,Coloring book moana,Art And Design,3.9,967.0,14.0,500000.0,Free,0.0,Everyone,2018-01-15,2.0.0,4.0.3 and up


### Combination of Markets

#### Data for Market Comparison
Merge the dataset based on the exactly same name.

- comp: combination of apple/android datasets by same App name

In [10]:
# extract some specific variable from data_android2 
comp = data_android2[['Name','iosname','Price','Size',
                            'Version','Rating','Type','Description','developer_address']]
# rename some variable 
comp = comp.rename({'Price':'Andro_price',
                                'Size':'Andro_size',
                                'Version':'Andro_version',
                                'Rating':'Andro_rating',
                                'Type':'Andro_type',
                                'Description':'Andro_description',
                                'Name': 'Andro_name',
                                'iosname':'Name'
                                }, axis='columns')
# combine it with those variables in data_apple
comp = pd.merge(comp, data_apple[['Name','Price','Category','Content_Rating',
                                              'Size','Version','Rating','Type','app_desc']], on=['Name'])
# rename new variables 
comp = comp.rename({'Price':'ios_price',
                                'Size':'ios_size',
                                'Version':'ios_version',
                                'Rating':'ios_rating',
                                'Type':'ios_type',
                                'app_desc':'Ios_description',
                                'Name':'ios_name'
                                }, axis='columns')
# change the order of the column 
comp = comp[['developer_address','Andro_name','ios_name','Category','Content_Rating',\
                         'Andro_price','ios_price','Andro_size','ios_size',\
                         'Andro_version','ios_version','Andro_rating','ios_rating',\
                         'Andro_type','ios_type','Andro_description','Ios_description']]
# choose those observations with the same name 
comp = comp[comp['Andro_name'] == comp['ios_name']]

comp.shape

(1547, 17)

In [11]:
# round the rating of android based on the 0.5 
def round_number(x, base):
    return base * round(float(x)/base)

comp['Price_Consistency'] = comp['Andro_price'] == comp['ios_price']
comp = comp.dropna(subset=['Andro_rating'])
comp['Andro_rating'] = comp['Andro_rating'].apply(lambda x: round_number(x, base=.5))
comp = comp.reset_index(drop = True)
comp.shape

(1540, 18)

In [14]:
data_android.to_csv('../data/clean_data-android.csv',index=None)

In [16]:
data_apple.to_csv('../data/clean_data-apple.csv',index=None)

In [20]:
apple_review2.to_csv('../data/clean_apple-review2.csv',index=None)

In [21]:
comp.to_csv('../data/clean_comp-markets.csv',index = None)

In [23]:
data_android2.to_csv('../data/clean_data-android2.csv',index=None)